Members: Adam Papini, Gary Burnett, Yousuf A. Khan

The code below is for problem #3

In [ ]:


library(readr)
install.packages("caret")

library(caret)





Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: lattice

Loading required package: ggplot2

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”


In [ ]:
install.packages("glmnet")
library(glmnet)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘shape’


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loaded glmnet 4.1-1



In [ ]:
train_data <- read_csv("train_data.csv")
test_data <- read_csv("test_data.csv")

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  date = col_date(format = ""),
  county = col_character()
)
ℹ Use `spec()` for the full column specifications.


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  date = col_date(format = ""),
  county = col_character()
)
ℹ Use `spec()` for the full column specifications.




In [ ]:

test_data <- subset(test_data, select = -c(X1))   #remove index col
train_data <- subset(train_data, select = -c(X1))

test_data$county <- as.factor(test_data$county)  #change country to levels 
train_data$county <- as.factor(train_data$county)


  


In [ ]:
log_loss <- function (data,
                      lev = NULL,
                       model = NULL) {
  out <- abs(log(1+data$obs) - log(1+data$pred)) 
  names(out) <- "MAD"
  out}

In [ ]:
#

train_data_new <-train_data[,!names(train_data) %in% c("county", "date", "response")]
train_data_new <- scale(train_data_new)

train_data_new <- data.frame(train_data_new, "response" = c(train_data$response))
train_data <- train_data_new

In [ ]:
# Define training control
set.seed(123)
library(tidyverse)
library(caret)
train.control <- trainControl(method = "repeatedcv", 
                              number = 10, repeats = 10)
# Train the model
model <- train(response ~., data = train_data, method = "lm",
               trControl = train.control)
# Summarize the results
print(model)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ tibble  3.1.0     ✔ dplyr   1.0.4
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ purrr   0.3.4     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ purrr::lift()   masks caret::lift()



Linear Regression 

18300 samples
   31 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 10 times) 
Summary of sample sizes: 16469, 16469, 16472, 16470, 16469, 16471, ... 
Resampling results:

  RMSE      Rsquared   MAE     
  14.31433  0.6471735  9.798148

Tuning parameter 'intercept' was held constant at a value of TRUE


In [ ]:
set.seed(123)
library(MLmetrics)
lambda <- 10^seq(-3, 3, length = 100)

log_loss <- function (data,
                        lev = NULL,
                        model = NULL) {

  out <- abs(log(1+data$obs) - log(1+data$pred)) 
  names(out) <- "MAD"
  out
}

robustControl <- trainControl(summaryFunction = madSummary)


earthFit <- train(response ~ .,
                  data = train_data, 
                  method = "lm",
                  metric = "MAD",
                  maximize = FALSE,
                  trControl = robustControl)
                  #tuneGrid = expand.grid(alpha = 0, lambda = 1.6))

#ridge <- train(
#  response ~., data = train_data, method = "glmnet", metric = "F1",
#  trControl = trainControl(summaryFunction = f1),
 #  tuneGrid = expand.grid(alpha = 0, lambda = lambda))

print(earthFit)

In [ ]:
set.seed(123)
lambda <- 10^seq(-3, 3, length = 100)
lasso <- train(
  response ~., data = train_data, method = "glmnet",
  trControl = trainControl("cv", number = 10),
  tuneGrid = expand.grid(alpha = 1, lambda = lambda)
  )
print(lasso)


Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


glmnet 

18300 samples
   31 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 16469, 16469, 16472, 16470, 16469, 16471, ... 
Resampling results across tuning parameters:

  lambda        RMSE      Rsquared   MAE      
  1.000000e-03  14.32314  0.6467688   9.802381
  1.149757e-03  14.32314  0.6467688   9.802381
  1.321941e-03  14.32314  0.6467688   9.802381
  1.519911e-03  14.32314  0.6467688   9.802381
  1.747528e-03  14.32314  0.6467688   9.802378
  2.009233e-03  14.32317  0.6467675   9.802365
  2.310130e-03  14.32327  0.6467626   9.802342
  2.656088e-03  14.32339  0.6467574   9.802306
  3.053856e-03  14.32365  0.6467458   9.802295
  3.511192e-03  14.32393  0.6467330   9.802303
  4.037017e-03  14.32430  0.6467153   9.802338
  4.641589e-03  14.32480  0.6466908   9.802395
  5.336699e-03  14.32541  0.6466610   9.802489
  6.135907e-03  14.32613  0.6466249   9.802635
  7.054802e-03  14.32703  0.6465805   9.802820
  8.111308e-03  14.32813  0.64652